In [ ]:
#pip install schedule
#pip install beautifulsoup4
#pip install requests
#pip install pandas

In [ ]:
pip install schedule

In [1]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
import schedule
import time
from datetime import datetime, timedelta


In [2]:
# Lista para armazenar os dados coletados
todos_dados = []

In [7]:
def map_status(img):
    """Mapeia o status da imagem para um valor numérico."""
    if img == 'bola_verde_P.png':
        return 2
    elif img == 'bola_amarela_P.png':
        return 1
    elif img == 'bola_vermelha_P.png':
        return 0
    return None

def extrair_dados():
    """Função para extrair e processar os dados."""
    # URL da página que contém os status dos serviços
    url = "http://www.nfe.fazenda.gov.br/portal/disponibilidade.aspx?versao=0.00&tipoConteudo=P2c98tUpxrI="
    
    # Criando uma sessão para manter o cookie
    session = requests.Session()
    
    # Realizando a requisição GET para obter o conteúdo da página
    response = session.get(url)
    
    # Verificando se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseando o conteúdo HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Encontrando a tabela que contém os dados desejados
        tabela = soup.find('table', {'id': 'ctl00_ContentPlaceHolder1_gdvDisponibilidade2'})
        
        # Extraindo as linhas da tabela
        linhas = tabela.find_all('tr')
        
        # Processando os dados de cada linha
        for linha in linhas[1:]:  # Pulando o cabeçalho
            colunas = linha.find_all('td')
            autorizador = colunas[0].text.strip()
            autorizacao4_img = colunas[1].find('img')['src'].split('/')[-1]
            status_servico_img = colunas[5].find('img')['src'].split('/')[-1]
            autorizacao4 = map_status(autorizacao4_img)
            status_servico = map_status(status_servico_img)
            todos_dados.append({
                'Timestamp': datetime.now(),
                'Autorizador': autorizador,
                'Autorizacao4': autorizacao4,
                'Status Serviço': status_servico
            })
        
        print(f"Dados coletados às {datetime.now().strftime('%H:%M:%S')}")
        verificar_problemas()
    else:
        print(f"Falha na requisição. Status code: {response.status_code}")

def verificar_problemas():
    """Função para verificar se algum serviço está com problemas (não verde)."""
    problemas = [dado for dado in todos_dados if dado['Autorizacao4'] != 2 or dado['Status Serviço'] != 2]
    
    if problemas:
        print("\nServiços com problemas:")
        for problema in problemas:
            autorizador = problema['Autorizador']
            status_autorizacao4 = "OK" if problema['Autorizacao4'] == 2 else "Problema"
            status_servico = "OK" if problema['Status Serviço'] == 2 else "Problema"
            print(f"- {autorizador}: Autorizacao4 = {status_autorizacao4}, Status Serviço = {status_servico}")
    else:
        print("\nTodos os serviços estão funcionando corretamente.")

def job():
    """Função agendada para ser executada a cada intervalo."""
    extrair_dados()

def salvar_csv():
    """Função para salvar os dados coletados em um arquivo CSV."""
    df = pd.DataFrame(todos_dados)
    timestamp_str = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Diretório onde você deseja salvar o arquivo CSV
    diretorio = os.path.expanduser("~/Downloads")  # Por exemplo, salva na pasta Downloads
    nome_arquivo = os.path.join(diretorio, f"status_servicos_{timestamp_str}.csv")
    
    df.to_csv(nome_arquivo, index=False)
    print(f"Dados salvos em '{nome_arquivo}'")

# Definindo o período de execução (3 minutos)
tempo_final = datetime.now() + timedelta(minutes=15)

# Agendar a execução do job a cada 1 minuto
schedule.every(5).minutes.do(job)

# Executar indefinidamente até o tempo final
while datetime.now() < tempo_final:
    schedule.run_pending()
    time.sleep(1)

# Salvar os dados coletados em um CSV após 3 minutos
salvar_csv()

Dados coletados às 19:48:06

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:48:06

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:48:06

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:48:07

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:49:07

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:49:07

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:49:07

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:49:07

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:50:08

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:50:08

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:50:08

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:50:08

Todos os serviços estão funcionando corretamente.
Dados coletados às 19:51:08

Todos os serviços estão